In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tarfile

In [ ]:
# inspecting tar file to understand what is in it
with tarfile.open("../data/gdc_download_20251211_092632.241106.tar.gz", "r:gz") as tar:
    tar.list()

-rw-r--r-- 0/0        227 2025-12-11 10:26:32 MANIFEST.txt 
-rw-r--r-- 0/0    4231705 2025-12-11 10:26:32 70f17418-6373-4928-ac14-64c61fa0650a/dcd92f62-bed1-4718-a918-6eda2855366a.rna_seq.augmented_star_gene_counts.tsv 


In [5]:
# extracting the tsv file containing gene counts 
tar_path = "../data/gdc_download_20251211_092632.241106.tar.gz"

with tarfile.open(tar_path, "r:gz") as tar:
    # Find the .tsv file inside the tar
    tsv_member = None
    for member in tar.getmembers():
        if member.name.endswith(".tsv"):
            tsv_member = member
            break

    if tsv_member is None:
        raise FileNotFoundError("No .tsv file found inside the tarball.")

    # Open it as a file-like object
    f = tar.extractfile(tsv_member)

    # Load as a dataframe
    df = pd.read_csv(f, sep="\t")

print("Loaded:", tsv_member.name)
print(df.head(n=20))
print(df.columns)


Loaded: 70f17418-6373-4928-ac14-64c61fa0650a/dcd92f62-bed1-4718-a918-6eda2855366a.rna_seq.augmented_star_gene_counts.tsv
                                                                                                                     # gene-model: GENCODE v36
gene_id            gene_name gene_type      unstranded stranded_first stranded_second tpm_unstranded fpkm_unstranded        fpkm_uq_unstranded
N_unmapped         NaN       NaN            1914040    1914040        1914040         NaN            NaN                                   NaN
N_multimapping     NaN       NaN            6954726    6954726        6954726         NaN            NaN                                   NaN
N_noFeature        NaN       NaN            20842156   73128082       21498083        NaN            NaN                                   NaN
N_ambiguous        NaN       NaN            6728690    155597         4215259         NaN            NaN                                   NaN
ENSG00000000003.15 TS

In [ ]:
# trying automatic extraction of files from TCGD GDC (probably need to put it a proper script)
## just copy pasted from chatgpt, needs to be modified
import requests

query = {
    "filters": {
        "op": "and",
        "content": [
            {"field": "cases.project.project_id", "value": "TCGA-GBM"},
            {"field": "files.access", "value": "open"}
        ]
    },
    "fields": "file_id",
    "format": "JSON"
}

url = "https://api.gdc.cancer.gov/files"
r = requests.post(url, json=query)
uuids = [item["file_id"] for item in r.json()["data"]["hits"]]
# write uuids to manifest format...
